In [1]:
import sys
sys.path.append('/kaggle/input/sentence-transformers')
sys.path.append('/kaggle/input/sentence-embedding-models')

import pandas as pd
import numpy as np
import os
import re
from tqdm import tqdm
import string
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
import torch
tqdm.pandas()

In [2]:
BASE_DIR = '/kaggle/input/learning-equality-curriculum-recommendations'

In [3]:
content_data = pd.read_csv(os.path.join(BASE_DIR,'content.csv'))
correlations_data = pd.read_csv(os.path.join(BASE_DIR,'correlations.csv'))
topics_data = pd.read_csv(os.path.join(BASE_DIR,'topics.csv'))
sub_data = pd.read_csv(os.path.join(BASE_DIR,'sample_submission.csv'))

In [4]:
content_data.shape,correlations_data.shape,topics_data.shape,sub_data.shape

((154047, 8), (61517, 2), (76972, 9), (5, 2))

In [5]:
model = SentenceTransformer('/kaggle/input/sentence-embedding-models/paraphrase-multilingual-mpnet-base-v2',device='cuda')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
)

In [6]:
sub_ids = sub_data['topic_id'].tolist()
topics_data_ = topics_data.query(f'id in {sub_ids}').reset_index(drop=True)

In [7]:
topics_data_['merged_text'] = topics_data_['title'].fillna('')+','+topics_data_['description'].fillna('')
content_data['merged_text'] = content_data['title'].fillna('')+','+content_data['description'].fillna('')


topics_data_['merged_text'].fillna('Not Available',inplace=True)
content_data['merged_text'].fillna('Not Available',inplace=True)

t_data = topics_data_[['id','merged_text']]
c_data = content_data[['id','merged_text']]

In [8]:
corpus_embeddings = model.encode(c_data['merged_text'], convert_to_tensor=True)

Batches:   0%|          | 0/4814 [00:00<?, ?it/s]

In [9]:
pred = []
top_k = 4
for query in t_data['merged_text']:
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    pred.append(top_results[1].cpu().numpy())

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
pred_final = []
for idx in pred:
    pid = c_data['id'][idx]
    pred_final.append(' '.join(pid))

In [11]:
sub_data.loc[:,'content_ids'] = pred_final
sub_data.to_csv('submission.csv',index=False)
sub_data.head()

,topic_id,content_ids
0,t_00004da3a1b2,c_0a78517e2aad c_070effcc5d44 c_8e9bb0117552 c...
1,t_00068291e9a4,c_20e1b5c2f49c c_4ab460948061 c_85e7c0954384 c...
2,t_00069b63a70a,c_a8f7827355a3 c_05dc1a4699de c_f44647ff1797 c...
3,t_0006d41a73a8,c_3c280717dabb c_fa21b549f383 c_c379a1e2a9a3 c...
4,t_4054df11a74e,c_f2d184a98231 c_51cb16d282df c_021c0d106065 c...
